In [2]:
import numpy as np
np.random.seed(0)



In [7]:
def generate_random_convex_function():
    x_points = np.random.randint(low=1, high=20, size=4)
    x_points.sort()
    x_points = np.append(0, x_points)  # the first 0 point is 0

    slopes = np.add.accumulate(np.random.random(size=3))
    slopes = np.append(0,slopes)  # the first slope is 0

    y_incr = np.ediff1d(x_points)*slopes
    y_points = np.add.accumulate(y_incr)
    y_points = np.append(0,y_points)  # the first y values is 0

    print(x_points)
    print(slopes)
    print(y_points)
    def final_function( x ):
        y = 0
        return y
    return final_function()

def generate_several_rcf():
    pass





In [8]:
for i in range(10):
    generate_random_convex_function()



[ 0  4  7  8 10]
[0.         0.67063787 0.88102043 1.00994673]
[0.         0.         2.01191361 2.89293404 4.9128275 ]
[ 0  4 11 13 15]
[0.         0.43860151 1.42697535 1.52902016]
[0.         0.         3.07021059 5.9241613  8.98220162]
[ 0  4  5  9 13]
[0.         0.41436859 0.88906609 1.51257619]
[ 0.          0.          0.41436859  3.97063295 10.02093772]
[ 0 14 16 17 18]
[0.         0.77834548 1.72791654 2.3904434 ]
[0.         0.         1.55669096 3.2846075  5.6750509 ]
[ 0  1  5 18 19]
[0.         0.09609841 1.07255787 1.54120907]
[ 0.          0.          0.38439363 14.32764598 15.86885505]
[ 0  8 10 14 17]
[0.         0.01998767 0.46169859 1.44128532]
[0.         0.         0.03997533 1.88676968 6.21062562]
[0 3 3 4 4]
[0.         0.69247212 1.25907357 1.52446306]
[0.         0.         0.         1.25907357 1.25907357]
[ 0  2  5 10 15]
[0.         0.9292962  1.24786515 1.91527553]
[ 0.          0.          2.78788859  9.02721434 18.60359199]
[ 0  1  1  7 17]
[0.         0